In [ ]:
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import torch

from heartbeat_detector.utils import seed_everything
from heartbeat_detector.models.unet_1d import UNet1d
from heartbeat_detector.dataset.dataset import HeartbeatDataloaders

In [ ]:
plt.rcParams['figure.figsize'] = [20, 7]

In [ ]:
seed_everything(420)
*__, test_dataloader = HeartbeatDataloaders(r'data\processed\2s_sin_with_channels\good_dataset.csv', 120, 4).get_train_validation_test_dataloaders()

In [ ]:
DEVICE = 'cuda'

In [ ]:
model_path = r"C:\Users\Ruslan\repos\lab\out\useful-asp-454\checkpoints\unet1d_epoch_040.pth"
model = torch.load(model_path).to(DEVICE)
model.eval();

In [ ]:
pred_batches = []
labels_bathes = []
signals_batches = []

with torch.no_grad():
    for signal_batch, label_batch in tqdm(test_dataloader, total=len(test_dataloader)):
        signal_batch = signal_batch.to(DEVICE)
        label_batch = label_batch.to(DEVICE)

        preds_batch = model(signal_batch)

        signals_batches.append(signal_batch.cpu().numpy().squeeze())
        pred_batches.append(preds_batch.cpu().numpy().squeeze())
        labels_bathes.append(label_batch.cpu().numpy().squeeze())

In [ ]:
preds = np.vstack(pred_batches).flatten()
labels = np.vstack(labels_bathes).flatten()
signals = np.vstack(signals_batches).flatten()

In [ ]:
record_no = 1000
record_len = 2  # in seconds
record_shift = record_no * record_len * 5000

start = record_shift
stop = 1500 + 1 + record_shift
alpha = 0.5

plt.plot(range(start, stop), signals[start:stop], 'b--', alpha=0.4, label='raw signal')
plt.plot(range(start, stop), labels[start:stop], 'g', alpha=alpha, label='ground truth')
plt.plot(range(start, stop), (preds)[start:stop], 'r', alpha=alpha, label='predictions')
plt.legend()
plt.ylim([-1.0, 1.0]);